## 1. 数组的排序

In [31]:
import numpy as np
# 简单的选择排序重复寻找列表中的最小值，并且不断交换直到列表是有序的
def selection_sort(x):
    for i in range(len(x)):
        swap = i + np.argmin(x[i:])
        (x[i], x[swap]) = (x[swap], x[i])
        return x
    
x = np.array([2, 1, 4, 3, 5])
selection_sort(x)

array([1, 2, 4, 3, 5])

### 1.1 NumPy中的快速排序：np.sort和np.argsort

np.sort默认是快速排序，另外也可以选择归并排序和堆排序，可以在不修改原始输入数组的基础上返回一个排好序的数组。

In [32]:
x = np.array([2, 1, 4, 3, 5])
np.sort(x)

array([1, 2, 3, 4, 5])

如果希望用排好序的数组替代原始数组，可以使用数组的sort方法

In [33]:
x.sort()
print(x)

[1 2 3 4 5]


另一个相关的函数是argsort，该函数返回的是原始数组排好序的索引值

In [34]:
x = np.array([2, 1, 4, 3, 5])
i = np.argsort(x)
print(i)

[1 0 3 2 4]


这些索引值可以被用于通过花哨的索引创建有序的数组

In [35]:
x[i]

array([1, 2, 3, 4, 5])

In [36]:
# 通过axis参数沿着行或列排序, 任何行列关系将丢失

rand = np.random.RandomState(42)
X = rand.randint(0, 10, (4, 6))
print(X)
print('对X的每列排序: \n', np.sort(X, axis = 0))
print('对X的每行排序: \n', np.sort(X, axis = 1))

[[6 3 7 4 6 9]
 [2 6 7 4 3 7]
 [7 2 5 4 1 7]
 [5 1 4 0 9 5]]
对X的每列排序: 
 [[2 1 4 0 1 5]
 [5 2 5 4 3 7]
 [6 3 7 4 6 7]
 [7 6 7 4 9 9]]
对X的每行排序: 
 [[3 4 6 6 7 9]
 [2 3 4 6 7 7]
 [1 2 4 5 7 7]
 [0 1 4 5 5 9]]


### 1.2 部分排序：分隔

有时候我们不希望对整个数组排序，仅仅希望找到数组中第K小的值，np.partition函数提供了这个功能，np.partiton函数的输入是数组和数字K，输出结果是一个新数组，最左边是第k小的值，往右是任意顺序的其他值

In [37]:
x = np.array([7, 2, 3, 1, 6, 5, 4])
np.partition(x, 3)

array([2, 1, 3, 4, 6, 5, 7])

返回数组的前三个值是数组中最小的三个值，剩下的位置是原始数组剩下的值，两个分隔区间中元素是任意排列的。与排序类似，也可以沿任意轴分隔。

In [38]:
print(X)
print('每列先返回最小两个值：\n', np.partition(X, 2, axis = 0))
print('每行先返回最小两个值：\n', np.partition(X, 2, axis = 1))

[[6 3 7 4 6 9]
 [2 6 7 4 3 7]
 [7 2 5 4 1 7]
 [5 1 4 0 9 5]]
每列先返回最小两个值：
 [[2 1 4 0 1 5]
 [5 2 5 4 3 7]
 [6 3 7 4 6 7]
 [7 6 7 4 9 9]]
每行先返回最小两个值：
 [[3 4 6 7 6 9]
 [2 3 4 7 6 7]
 [1 2 4 5 7 7]
 [0 1 4 5 9 5]]


### 1.3 示例：K个最近邻

以下示例展示如何使用argsort函数沿着多个轴快速找到集合中每个点的最近邻。

In [39]:
# 首先在二维平面上创建一个有10个随机点的集合，按照惯例这些数据点放在一个10X2的数组中
X = np.random.rand(10, 2)
print('X: \n', X)

# 将这些点画散点图
import matplotlib.pyplot as plt
import seaborn; seaborn.set # 设置绘图风格

plt.scatter(X[:, 0], X[:, 1], s = 100)

X: 
 [[0.96193638 0.29214753]
 [0.24082878 0.10029394]
 [0.01642963 0.92952932]
 [0.66991655 0.78515291]
 [0.28173011 0.58641017]
 [0.06395527 0.4856276 ]
 [0.97749514 0.87650525]
 [0.33815895 0.96157015]
 [0.23170163 0.94931882]
 [0.9413777  0.79920259]]


<Figure size 432x288 with 1 Axes>

现在来计算两两数据点对间的距离，两点间的距离的平方等于每个维度距离差的平方的和，利用广播和聚合功能计算矩阵的平方距离。

In [40]:
dist_sq = np.sum((X[:,np.newaxis,:] - X[np.newaxis,:,:]) **2, axis = -1)
dist_sq.shape

(10, 10)

In [41]:
# 确认以上步骤，应该看到该矩阵的对角线（也就是每个点到其自身的距离）都是0
dist_sq.diagonal()

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [42]:
# 得到一个转化为两点间的平方距离的矩阵后，就可以使用np.argsort函数沿着每行进行排序，最左边的列给出的索引值就是最近邻
# 需要注意的是，第一列是按照0-9从小到大顺序排列的，这是因为每个点的最近邻是其自身
nearest = np.argsort(dist_sq, axis = 1)
nearest

array([[0, 9, 3, 6, 4, 1, 7, 5, 8, 2],
       [1, 5, 4, 0, 3, 8, 2, 7, 9, 6],
       [2, 8, 7, 4, 5, 3, 1, 9, 6, 0],
       [3, 9, 6, 7, 4, 8, 0, 2, 5, 1],
       [4, 5, 8, 7, 2, 3, 1, 9, 0, 6],
       [5, 4, 1, 2, 8, 7, 3, 0, 9, 6],
       [6, 9, 3, 0, 7, 8, 4, 2, 5, 1],
       [7, 8, 2, 3, 4, 5, 9, 6, 1, 0],
       [8, 7, 2, 4, 3, 5, 9, 6, 1, 0],
       [9, 6, 3, 0, 7, 4, 8, 5, 2, 1]])

如果使用全排序，我们实际上可以实现的比这个例子展示的更多。如果仅仅关心k个最近邻，那么唯一需要做的是分隔每一行，这样最小的k + 1的平方距离将排在最前面，其他更长距离占据矩阵该行的其他位置。可以用np.argpartition函数实现：

In [43]:
k = 2
nearest_partition = np.argpartition(dist_sq, k + 1, axis = 1)
nearest_partition

array([[0, 9, 3, 6, 4, 1, 7, 5, 8, 2],
       [1, 4, 5, 0, 3, 2, 6, 7, 8, 9],
       [2, 8, 7, 4, 5, 3, 1, 9, 6, 0],
       [3, 9, 6, 7, 4, 5, 1, 2, 8, 0],
       [4, 8, 5, 7, 1, 2, 3, 9, 6, 0],
       [4, 5, 1, 2, 8, 7, 3, 0, 6, 9],
       [3, 9, 6, 0, 1, 5, 2, 7, 8, 4],
       [8, 7, 2, 3, 4, 5, 9, 6, 1, 0],
       [8, 7, 2, 4, 3, 5, 9, 6, 1, 0],
       [3, 9, 6, 0, 1, 5, 2, 7, 8, 4]])

In [44]:
# 为了将邻节点网络可视化，我们将每个点和其最近的两个最近邻连接
plt.scatter(X[:, 0], X[:, 1], s = 100)

k = 2
for i in range(X.shape[0]):
    for j in nearest_partition[i, :k+1]:
        # 画一条从X[i]到X[j]的线段
        # 用zip方法实现
        plt.plot(*zip(X[j], X[i]), color = 'black')

<Figure size 432x288 with 1 Axes>

## 2. 结构化数据：NumPy的结构化数组

假定现在有关于一些人的分类数据（如姓名，年龄和体重），我们需要存储这些数据用于Python项目，一种可行的方法是将它们存在三个单独的数组中

In [45]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68, 61.5]

但这种方法有点笨，因为并没有任何信息能说明这三个数组之间是相关联的，可以用结构化数组来存储所有数据

In [46]:
# 生成一个简单的数组
x = np.zeros(4, dtype = int)
print('x: ', x)

# 通过指定复合数据结构，构造一个结构化数组，dtype参数是一个字典，‘name’指定字段名称每个元素包含三个字段：name, age, weight
data = np.zeros(4, dtype = {'names':('name', 'age', 'weight'),
                            'formats':('U10', 'i4', 'f8')})
# u10:表示长度不超过10的Unicode字符串，i4表示4字节整型，f8表示8字节浮点型
print('data.dtype:', data.dtype)
print('空数组data:', data)

# 将列表数据放入生成的空数组中
data['name'] = name
data['age'] = age
data['weight'] = weight
print('data: ', data)

# 通过索引或名称查看相应的值
print('获取所有名称：', data['name'])
print('获取数据第一行：', data[0])
print('获取最后一行的名字：', data[-1]['name'])

# 利用布尔掩码复杂操作，例如按照年龄进行筛选
print('获取年龄小于30岁的人信息', data[data['age'] < 30])
print('获取年龄小于30岁的人的名字', data[data['age'] < 30]['name'])

x:  [0 0 0 0]
data.dtype: [('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]
空数组data: [('', 0, 0.) ('', 0, 0.) ('', 0, 0.) ('', 0, 0.)]
data:  [('Alice', 25, 55. ) ('Bob', 45, 85.5) ('Cathy', 37, 68. )
 ('Doug', 19, 61.5)]
获取所有名称： ['Alice' 'Bob' 'Cathy' 'Doug']
获取数据第一行： ('Alice', 25, 55.)
获取最后一行的名字： Doug
获取年龄小于30岁的人信息 [('Alice', 25, 55. ) ('Doug', 19, 61.5)]
获取年龄小于30岁的人的名字 ['Alice' 'Doug']


### 2.1 生成结构化数组

采用`字典方式`，制定结构化数组的数据类型

In [47]:
np.dtype({'names':('name', 'age', 'weight'),
                   'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

数值数据类型可以用Python类型或NumPy的dtype类型指定

In [48]:
np.dtype({'names':('name', 'age', 'weight'),
                   'formats':((np.str_, 10), int, np.float32)})

dtype([('name', '<U10'), ('age', '<i8'), ('weight', '<f4')])

采用`元组列表方式`，制定结构化数组的数据类型

In [49]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

如果类型名称并不重要可以省略，仅仅指定数据类型

In [50]:
np.dtype('S10, i4, f8')

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

| NumPy数据类型符号| 描述                                    | 示例                             |  
| ---------        | -----------                            | -------                             |  
| ``'b'``          | 字节型                                 | ``np.dtype('b')``                   |  
| ``'i'``          | 有符号整型                            | ``np.dtype('i4') == np.int32``      |  
| ``'u'``          | 无符号整型                           | ``np.dtype('u1') == np.uint8``      |  
| ``'f'``          | 浮点型                              | ``np.dtype('f8') == np.int64``      |  
| ``'c'``          | 复数浮点型                          | ``np.dtype('c16') == np.complex128``|  
| ``'S'``, ``'a'`` | 字符串                             | ``np.dtype('S5')``                  |  
| ``'U'``          | Unicode编码字符串                  | ``np.dtype('U') == np.str_``        |  
| ``'V'``          | 原生数据 Raw data (空，void)       | ``np.dtype('V') == np.void``        |

### 2.2 更高级的复合类型

之前创建的每个元素例如name, age都是一个`列表`，也可以创建更高级的复合类型比如每个元素是一个`数组或矩阵` 

In [51]:
# 创建一个只包含1个元素的结构化数组，其中该元素的数据类型是一个 mat组件包含一个3x3的浮点矩阵
user_id = [1, 2, 3, 4]
X = np.zeros(4, dtype = ([('user_id', 'i8'), ('mat', 'f8', (3, 3))]))
X['user_id'] = user_id
print('X: ', X)
print('获取第一行：', X[0])
print('mat:', X['mat'])
print('mat第一层：', X['mat'][0])

X:  [(1, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
 (2, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
 (3, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
 (4, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])]
获取第一行： (1, [[0., 0., 0.], [0., 0., 0.], [0., 0., 0.]])
mat: [[[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]

 [[0. 0. 0.]
  [0. 0. 0.]
  [0. 0. 0.]]]
mat第一层： [[0. 0. 0.]
 [0. 0. 0.]
 [0. 0. 0.]]


### 2.3 记录数组：结构化数组的扭转

Numpy提供了`np.recarray`类，与结构化数组几乎相同，但有一个独特特征：域可以像属性一样获取，而不是像字典的键那样获取。

In [52]:
# 类似字典的键获取方式
data['age']

array([25, 45, 37, 19], dtype=int32)

如果将这些数据当作一个记录数组，可以用很少的键来获取这个结果, 可以使用点操作符访问字段,每个字段代表一列。

In [53]:
# data.view 允许你创建一个与原始结构化数组共享数据存储的视图，从而可以以不同的方式查看和操作相同的数据。这在某些情况下可以提高效率，因为不需要复制数据。
# 但要注意，修改视图可能会影响原始数组，因此需要小心操作。
data_rec = data.view(np.recarray)
data_rec.age

array([25, 45, 37, 19], dtype=int32)

## 3. 闯关题

### STEP1：根据要求完成题目

#### Q1：找出最小的三个值  
利用随机种子0生成一个包含1000000个元素的随机浮点数数组 data，数值范围从0到1。  
利用np.partition找出前三小的值，它们是（）。  
A、7.07120317e-07, 1.33367964e-06, 2.95041628e-06  
B、3.31055446e-06, 3.76517081e-06, 6.04484695e-06  
C、6.78447358e-06, 8.28558723e-06, 1.17269413e-05  
D、8.35490034e-06, 1.35423757e-05, 1.62557428e-05  

In [54]:
# 数组创建
np.random.seed(0)
data = np.random.rand(1000000)
print(np.partition(data,3))

[7.07120317e-07 1.33367964e-06 2.95041628e-06 ... 6.45894113e-01
 6.02763376e-01 9.08605884e-01]


In [55]:
#填入你的答案并运行,注意大小写
a1 = 'A'  # 如 a1= 'A'

#### Q2:   寻找最优配送点  
你正在为一个物流公司工作，该公司希望优化其在城市中的配送中心的位置。公司的目标是最小化配送路径的总距离，从而降低成本和提高效率。你有一组包含100个配送地点的坐标数据，这些数据模拟了城市中的配送需求。  

条件：  

1.	利用随机种子0生成一个形状为 (100, 2) 的数组 locations，代表城市中的100个配送地点的坐标，均值为[50, 50]，协方差矩阵为[[100, 50], [50, 100]]。  

要求：  

1.	数据分析：  
使用NumPy计算所有配送地点间的欧几里得距离矩阵。参考：  
```
dist_sq = np.sum((locations[:, np.newaxis, :] - locations[np.newaxis, :, :]) ** 2, axis=-1)  
distances = np.sqrt(dist_sq)  
```

2.	优化配送中心位置：  
找出与其他所有配送地点平均距离最小的配送地点，作为新的配送中心。  
将配送中心的索引保存为a2

In [56]:
# 创建坐标
np.random.seed(0)
locations = np.random.multivariate_normal([50,50], [[100,50],[50,100]], 100)
dist_sq=np.sum((locations[:,np.newaxis,:]-locations[np.newaxis,:,:])**2,axis=-1)
distances = np.sqrt(dist_sq)
A2 = np.mean(distances,axis=1)
a2 = np.argmin(A2)

In [57]:
#填入你的答案并运行,注意大小写
a2 = 28  # 如 a2= 50

#### Q3：处理结构化数据  
根据已给的结构化数据，计算公司员工的平均工资。结果保留整数。

In [58]:
dtype = [('name', 'U10'),  ('salary', float)]

employees = np.array([
    ('Alice', 50000.00),
    ('Bob',  55000.00),
    ('Charlie',  65000.00),
    ('David',  48000.00),
    ('Eve',  62000.00)
], dtype=dtype)
employees['salary'].mean()

56000.0

In [59]:
#填入你的答案并运行,注意大小写
a3 = 56000 # 如 a3= 200

### STEP2：将结果保存为 csv 文件  
csv 需要有两列，列名：id、answer。其中，id 列为题号，如 q1、q2；answer 列为 STEP1 中各题你计算出来的结果。💡 这一步的代码你不用做任何修改，直接运行即可。

In [60]:
# 生成 csv 作业答案文件
def save_csv(a1,a2,a3):
    import pandas as pd
    df = pd.DataFrame({"id": ["q1", "q2", "q3"], "answer": [a1, a2, a3]})
    df.to_csv("answer_4.csv", index=None)

save_csv(a1,a2,a3)  # 运行这个cell,生成答案文件；该文件在左侧文件树project工作区下，你可以自行右击下载或者读取查看

### STEP3: 提交 csv 文件，获取分数结果  
你的 csv 答案文件已经准备完毕了，最后让我们提交答案文件，看看是否正确。  
提交方法：  
1、拷贝提交 token  
去对应关卡的 [提交页面](https://www.heywhale.com/home/competition/667278de2c79f2f6c008995b/submit)，找到对应关卡，看到了你的 token 嘛？  
拷贝它。  
记得：每个关卡的 token 不一样。  
2、下方 cell 里，拿你拷贝的 token 替换掉 XXXXXXX， 然后 Ctrl+Enter 运行 

In [61]:
#运行这个Cell 下载提交工具

!wget -nv -O heywhale_submit https://cdn.kesci.com/submit_tool/v4/heywhale_submit&&chmod +x heywhale_submit

# 运行提交工具
# 把下方XXXXXXX替换为你的 Token
# 改完看起来像是：!./heywhale_submit -token 586eeef71cb92941 -file answer_4.csv

!./heywhale_submit -token 59a57d5102ac47a9 -file answer_4.csv  # 替换XXXXXXX；注意不可增减任何空格或其他字符

wget: /opt/conda/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /opt/conda/lib/libssl.so.1.0.0: no version information available (required by wget)
2025-10-26 14:27:39 URL:https://cdn.kesci.com/submit_tool/v4/heywhale_submit [22020102/22020102] -> "heywhale_submit" [1]
Heywhale Submit Tool: v5.0.1 
> 已验证Token
> 开始上传文件
30 / 30 [||||||||||||||||||||||||||||||||||||||||||||||||||||||||] ? p/s 100.00%
> 文件已上传        
> 服务器响应: 200 提交成功，请等待评审完成
> 提交完成
